In [2]:
#Import All Important Libraries
# import librosa
# from pydub import AudioSegment
import subprocess
import collections as coll
# import soundfile
import pandas as pd
import os, glob#, pickle
# import numpy as np
# import soundfile
import os, glob#, pickle
from collections import defaultdict
# import numpy as np
# import matplotlib.pyplot as plt
# from sklearn.model_selection import train_test_split
# from sklearn.neural_network import MLPClassifier
# from sklearn.metrics import accuracy_score, confusion_matrix

In [4]:
d_wav=coll.defaultdict(int)
d_txt=coll.defaultdict(int)
m = './Datasets/'
folder = m+"JL Corpus"
for count, filename in enumerate(os.listdir(folder)):
    s = filename.split('_') 
    if filename[-3:]=='wav':
       dst = f"{s[1]} {str(d_wav[s[1]])}.wav"
       d_wav[s[1]]+=1
    else:
        dst = f"{s[1]} {str(d_txt[s[1]])}.txt"
        d_txt[s[1]]+=1
    
    src =f"{folder}/{filename}"  # foldername/filename, if .py file is outside folder
    dst =f"{folder}/{dst}"
    # print(src,dst)
    os.rename(src, dst)

In [ ]:
# Convert to WAV
target = "rv1"
for folder in glob.glob(f'./datasets/{target}/*'):
    print(folder)
    for file in glob.glob(folder + '/*.wav'):
            # print(file)
        file_name = os.path.basename(file)
        # opfile = "./test/test_anger_4.wav"
        # if file[-3:] != "wav":
        cmd = f"ffmpeg -i {file_name} -acodec pcm_s16le -ac 1 -ar 48000 {file_name} -y" # convert to wav
        cmd = f"ffmpeg -i {file_name} -ac 1 {file_name} -y" # convert wav to monophonic
        process = subprocess.Popen(cmd.split(), stdout=subprocess.PIPE)
        output, error = process.communicate()
        # else:
        #     file_name = file
        # file_name = os.path.basename(file_name)

In [ ]:
# Copy files
source = "ravdess"
dest = "./datasets/ravdess/"
for folder in glob.glob(f'./datasets/{source}/*'):
    print(folder)
    for file in glob.glob(folder + '/*.wav'):
        file_name = os.path.basename(file)
        cmd = f"cp -lr {file} {dest}" # convert wav to monophonic
        process = subprocess.Popen(cmd.split(), stdout=subprocess.PIPE)
        output, error = process.communicate()
        # else:
        #     file_name = file
        # file_name = os.path.basename(file_name)

In [ ]:
df = pd.read_csv('./datasets_og/meld/train/train_sent_emo.csv')
emodict = {}
filelist = coll.defaultdict(int)
namedict = {'neutral': 0, 'surprise': 0, 'fear': 0, 'sad': 0, 'happy': 0, 'disgust': 0, 'anger': 0}
for i, r in df.iterrows():
    emodict[(r['Dialogue_ID'], r['Utterance_ID'])] = (r['Emotion'], r['Sentiment'])
# print(emodict)
print(len(emodict))
for file in glob.glob("./datasets/meld/*.wav"):
    filelist[os.path.basename(file)] = 1
print(len(filelist))
for file in glob.glob("./datasets_og/meld/train/train_splits/*.mp4"):
    file_name = os.path.basename(file)
    # print(file_name[:-4])
    i, j = file_name[:-4].split("_")
    # print(i, j)
    i, j = emodict[(int(i[3:]), int(j[3:]))]
    if i == 'sadness':
        i = 'sad'
    elif i == 'joy':
        i = 'happy'
    # print(i, j)
    out_name = f"{i}_{j}_{namedict[i]}.wav"
    if out_name in filelist:
        # print(out_name[:-4])
        namedict[i] = int(out_name.split("_")[-1][:-4])+1
        continue
    namedict[i] += 1
    dest = f"./datasets/meld/{out_name}"
    cmd = f"ffmpeg -i {file} -vn -acodec pcm_s16le -ar 44100 -ac 2 {dest}"
    process = subprocess.Popen(cmd.split(), stdout=subprocess.PIPE)
    output, error = process.communicate()
    # print(file, file_name, out_name, dest)

In [ ]:
for file in glob.glob("./datasets/meld/*.wav"):
    file_name = os.path.basename(file)
    print(file_name)
    i = file_name[:-4].split("_")
    # print(i, j, k)
    out_name = f"{i[0]}_{i[-1]}.wav"
    dest = f"./datasets/meld/{out_name}"
    cmd = f"mv {file} {dest}"
    # print(file, dest)
    process = subprocess.Popen(cmd.split(), stdout=subprocess.PIPE)
    output, error = process.communicate()
    # break
    # print(file, file_name, out_name, dest)